<center> <h1> <b>BIRDSONGS<b></h1> </center>
<center> <h1>Syllable Tutorial</h1> </center>

# Define Objects

Import package and define matplotlib backend

In [ ]:
# the following line enable interact with figures, 
# you can make zoom and save images from a poup matplotlib window
# to select points: qt ipympl tk. To just visualize: notebook
%matplotlib ipympl

import birdsongs as bs
from birdsongs.utils import *

Define directories path and show audio files

In [ ]:
# root_path   = "path_to_repository\\'
# audios_path = "audios_path\\'
# bird_name   = "Zonotrichia capensis"
# audios_path = "C:\\Users\\sebas\\Documents\\GitHub\\audios\\Dissertation-xeno\\"#rufous-collared-sparrow\\humbolt\\"

paths  = bs.Paths()              # root_path, audios_path, bird_name
ploter = bs.Ploter(save=True)   # to save figures save=True 
paths.AudioFiles()

Define, visualize and hear birdsong. The same funcion allow to us select the time of interest enabling SelectTime_on option

In [ ]:
birdsong = bs.BirdSong(paths, no_file=2, NN=512, flim=(1e3,20e3), tlim=(0.,5.),
                       Nt=5000, umbral=0.05, split_method="amplitud")
ploter.Plot(birdsong, FF_on=False, SelectTime_on=True)
AudioPlay(birdsong)

In [ ]:
#time_interval = Positions(ploter.klicker)[0]
time_interval = np.array([2.842465632897991, 2.9563384444937864])
print("Time interval selected: initial time {:.4f} s, final time {:.4f} s".format(time_interval[0], time_interval[1]))

In [ ]:
syllable = bs.Syllable(birdsong, tlim=time_interval, umbral_FF=1.5, NN=birdsong.NN, Nt=30, 
                        ide="syllable", no_syllable=2)
ploter.Plot(syllable, FF_on=True)
AudioPlay(syllable)

In [ ]:
#plt.figure(); plt.plot(syllable.FF, 'o'); plt.show(); plt.ylim((0, 2e4));
#syllable.FF

3D plot (in process)

In [ ]:
# ploter.Plot3d(birdsong)

In [ ]:
# ploter.Plot(birdsong, FF_on=False, syllable=syllable)

Define syllable with the default syllable extractor

In [ ]:
# no_syllable    = int(input("Enter the syllable number (1 to {0}): ".format(bird.no_syllables)))
# syllable       = birdsong.Syllable(no_syllable=3)
# ploter.Plot(syllable)
# AudioPlay(syllable)

In [ ]:
# ploter.Plot(birdsong, FF_on=False, syllable_on=True, chunck_on=False)

# Optimal Synthetic Syllable

Define optimization method and its parameters 

In [ ]:
brute     = {'method':'brute', 'Ns':11}  # , "workers":-1
optimizer = bs.Optimizer(syllable, method_kwargs=brute)
#?lmfit.minimize

Find optimal $\gamma$ for all the syllables computed by the syllables extractor. You can also just find the optimal for the single syllable

In [ ]:
# optimal_gamma = optimizer.OptimalGamma(syllable)
optimal_gamma = 37000 # optimizer.AllGammas(birdsong)
syllable.p["gm"].set(value=optimal_gamma)

In [ ]:
syllable.p

Optimal $\alpha$ and $\beta$

In [ ]:
#              [a0, a1, a2, b0, b1, b2]
# syllable.Set([0.01, 0, 0, -0.25, 1, 2])
# syllable.p["b2"].set(value=2)

In [ ]:
optimizer.OptimalParams(syllable, Ns=5)

Display optimal parameters and generate a synthetic syllable 

In [ ]:
Display(syllable.p)
synth_syllable = syllable.Solve(syllable.p)

Visualize and plot the synthetic syllable

In [ ]:
ploter.PlotVs(synth_syllable);

In [ ]:
ploter.PlotAlphaBeta(synth_syllable);

In [ ]:
ploter.Result(syllable, synth_syllable);

In [ ]:
ploter.Syllables(syllable, synth_syllable);

In [ ]:
AudioPlay(syllable)

In [ ]:
AudioPlay(synth_syllable)

Write synthetic and real syllables

In [ ]:
syllable.WriteAudio();  synth_syllable.WriteAudio();